# (1) Importing/Storing data

In [1]:
import wfdb
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
### sampled at 360 samples per second

In [3]:
records_A = []
properties_A = []
annot_A = []
AnnSymb_A = []
AnnSamp_A = []
AnnRhythm_A = []
Rpeak_Samp_A = []
Rpeak_Symb_A = []


for f in glob.glob('C:/Users/masud/Skrivebord/Github_Bachelor_Project/wfdb-python/data/arrhythmia_data/*.dat'): ##### change the path to own directory.       
    sig, fields = wfdb.rdsamp(f[:-4], channels=[1])  #### In this function, pass "channels=[0]" or "channels=[1]" to select channel 1 or 2.
    
    ann = wfdb.rdann(f[:-4], 'atr')
    QRS = wfdb.rdann(f[:-4], 'xws')
    Symb = pd.Series(ann.symbol)
    Samp = pd.Series(ann.sample)
    
    QRS_Symb = pd.Series(QRS.symbol)
    QRS_Samp = pd.Series(QRS.sample)
    Rhythm = pd.Series(ann.aux_note)
    records_A.append(sig)
    properties_A.append(fields)
    annot_A.append(ann)
    AnnSymb_A.append(Symb)
    AnnSamp_A.append(Samp)
    Rpeak_Symb_A.append(QRS_Symb)
    Rpeak_Samp_A.append(QRS_Samp)
    AnnRhythm_A.append(Rhythm)
    
    

AnnSymb_A = pd.Series(AnnSymb_A).values
AnnSamp_A = pd.Series(AnnSamp_A).values

In [4]:
AnnSymb_A[24] #Rpeak_Symbol

0       +
1       N
2       N
3       N
4       N
       ..
2034    N
2035    a
2036    N
2037    x
2038    N
Length: 2039, dtype: object

In [5]:
AnnSamp_A[24] #Rpeak_Sample

0           60
1          159
2          415
3          686
4          905
         ...  
2034    648419
2035    648594
2036    649104
2037    649181
2038    649800
Length: 2039, dtype: int32

In [6]:
AnnRhythm_A[24]

0       (AFIB 
1             
2             
3             
4             
         ...  
2034          
2035          
2036          
2037          
2038          
Length: 2039, dtype: object

In [7]:
# Rpeak_Symb_A[47] ###Annotaion_Symbol
# print(len(Rpeak_Symb_A[47])) ### length = 43

In [8]:
# Rpeak_Samp_A[24] ### Annotaion_Sample
# print(len(Rpeak_Samp_A[47])) ### length = 43

In [9]:
# df = pd.DataFrame(AnnRhythm_A[24]) ### length = 2764
# df
# df.to_excel("AnnRhythm_A[24].xlsx") ### output an excel file

# (2) Pre processing data

In [10]:
appended_AnnSamp = [item for i in range(len(AnnSamp_A)) for item in AnnSamp_A[i]]
appended_AnnSymb = [item for i in range(len(AnnRhythm_A)) for item in AnnRhythm_A[i]]

In [11]:
df = pd.DataFrame(
    {'Rpeaks': appended_AnnSamp,
     'Symbol': appended_AnnSymb
    })

df

,Rpeaks,Symbol
0,18,(N 
1,77,
2,370,
3,662,
4,946,
...,...,...
112642,648797,
112643,649040,
112644,649292,
112645,649536,


In [12]:
df = df.replace(r'^\s*$', np.nan, regex=True) ### Replace empty places with NaN
df

,Rpeaks,Symbol
0,18,(N 
1,77,NaN
2,370,NaN
3,662,NaN
4,946,NaN
...,...,...
112642,648797,NaN
112643,649040,NaN
112644,649292,NaN
112645,649536,NaN


In [13]:
df = df.fillna(method='ffill') ### Perform "forward fill" - filling instances (NaN) in front row with preceeeding value
df

,Rpeaks,Symbol
0,18,(N 
1,77,(N 
2,370,(N 
3,662,(N 
4,946,(N 
...,...,...
112642,648797,(N 
112643,649040,(N 
112644,649292,(N 
112645,649536,(N 


In [14]:
final_df = df[(df['Symbol'] == '(N') | (df['Symbol'] == '(AFIB')]
final_df

,Rpeaks,Symbol
17671,23,(N
17672,88,(N
17673,442,(N
17674,789,(N
17675,1155,(N
...,...,...
95413,597713,(AFIB
95414,597904,(AFIB
95415,598044,(AFIB
95416,598210,(AFIB
